# Setup

Collab specific stuff:

In [1]:
running_colab = False

In [2]:
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    logs_directory = 'drive/MyDrive/summaries'
    !pip install -r "drive/MyDrive/requirements.txt"
else:
    logs_directory = 'summaries'

Imports

In [3]:
import time
import gym_2048
from tensorforce import Agent, Environment
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext tensorboard

Functions

# Useful code snippets

Render "beautiful" 2048 grid:

`imshow(environment._environment.environment.render(mode="rgb_array"))`

Illegal move reward:

`environment._environment.environment.set_illegal_move_reward(-10)`

Log2 reward:

`log2_reward = reward if reward <= 0 else np.log2(reward)`

Show TensorBoard graphs:

`%tensorboard --logdir summaries` (with parameter `summarizer=dict(directory='summaries')` in the `Agent.create()` method)

# Hyperparams to test

`exploration`

`learning_rate`

Negative reward for illegal moves: `environment._environment.environment.set_illegal_move_reward(-1)`

`target_sync_frequency`

`batch_size` & `update_frequency`

Constant reward / Log reward

# Custom hyper params train

Modified hyperparams :

Ex: `learning_rate = 0.01` et `batch_size = 32`

In [4]:

hyperparams_dict = {
    'max_episode_timesteps': 1000 # Maximum steps allowed in episode. This hyper-param is for environment
    , 'batch_size': 16
    , 'update_frequency': 4
    , 'learning_rate': 0.001
    , 'discount': 0.8
    , 'memory': 30000
    , 'exploration': 0.1
    , 'target_sync_frequency': 4
    , 'horizon': 2
}

def train_agent(params, num_episodes, print_freq, csv_name):
    ''' Function creating environment & agent based on params dict. 
    num_episodes defines how many episodes agent will train & print_freq defines 
    printing frequency. 
    Function will create dict of metrics & dict of params used in function & both
    will be saved as a csv with csv_name'''

    # create a 2048 environment
    environment = Environment.create(
        environment = 'gym'
        , level = '2048-v0'
        , max_episode_timesteps = hyperparams_dict['max_episode_timesteps']
    )

    # create an agent
    agent = Agent.create(
        agent = 'dqn'
        , batch_size = hyperparams_dict['max_episode_timesteps']
        , update_frequency = hyperparams_dict['update_frequency']
        , environment = environment
        , learning_rate = hyperparams_dict['learning_rate']
        , discount = hyperparams_dict['discount']
        , memory = hyperparams_dict['memory']
        , exploration = hyperparams_dict['exploration']
        , target_sync_frequency = hyperparams_dict['target_sync_frequency']
        , horizon = hyperparams_dict['horizon']
        , summarizer = dict(directory=logs_directory
                            , summaries=[
                                "regularization-loss"
                                , "loss"
                                , "episode-length"
                                , "episode-reward"
                                , "objective-loss"
                                , "reward"
                                , "update-return"
                            ]
                           )
    )

    # instantiate lists to record global training metrics
    max_tiles = []
    scores = []
    updates = []
    valid_moves = []
    run_time = []
    start_training_time = time.time()

    for episode in range(num_episodes):
        state = environment.reset()
        terminal = False

        #Checking metrics while training
        state_freeze = state.copy()
        num_updates = 0
        num_moves = 0
        invalid_moves = 0
        start_episode_time = time.time()

        while not terminal:
            #Core
            action = agent.act(states=dict(state=state,  action_mask=environment._environment.environment.get_invalid_moves()))
            state, terminal, reward = environment.execute(actions=action)
            log2_reward = reward if reward <= 0 else np.log2(reward)
            num_updates += agent.observe(terminal=terminal, reward=log2_reward)

            #Number of moves
            num_moves += 1

            #Number of invalid moves
            if (state == state_freeze).all():
                invalid_moves += 1
            state_freeze = state.copy()

        # Storing score and max tile
        max_tiles.append(environment._environment.environment.Matrix.max())
        scores.append(environment._environment.environment.score)
        updates.append(num_updates)
        valid_moves.append(num_moves)
        run_time.append(round(time.time() - start_episode_time,2))

        if episode % print_freq == 0:
            print('Episode {}: score = {}, terminal = {}, updates={}, max_tile={}, valid_moves={}, invalid_moves={}, seconds={}'\
                  .format(episode, environment._environment.environment.score , terminal, num_updates, max_tiles[-1], num_moves-invalid_moves, invalid_moves, round(time.time() - start_episode_time,2)))
        
    metrics_dict = {
        'max_tiles': max_tiles
        , 'scores': scores
        , 'updates': updates
        , 'valid_moves': valid_moves
        , 'run_time' : run_time
    }
    
    params_dict = params
    params_dict['num_episodes'] = num_episodes
    
    #params_df = pd.DataFrame(params_dict)
    metrics_df = pd.DataFrame(metrics_dict)
    
    #params_path = f'training_csv/params_{csv_name}.csv'
    metrics_path = f'training_runs/{csv_name}_metrics.csv'
    #params_df.to_csv(params_path)
    metrics_df.to_csv(metrics_path)
    
    environment.close()
    
    return metrics_df

In [ ]:
test_df = train_agent(hyperparams_dict, 1000, 100, '1000_test')

Episode 0: score = 660.0, terminal = 1, updates=20, max_tile=64, valid_moves=87, invalid_moves=0, seconds=5.04
Episode 100: score = 684.0, terminal = 1, updates=23, max_tile=64, valid_moves=93, invalid_moves=0, seconds=2.22
Episode 200: score = 1492.0, terminal = 1, updates=37, max_tile=128, valid_moves=150, invalid_moves=0, seconds=3.57
Episode 300: score = 664.0, terminal = 1, updates=22, max_tile=64, valid_moves=91, invalid_moves=0, seconds=1.99
Episode 400: score = 864.0, terminal = 1, updates=25, max_tile=64, valid_moves=101, invalid_moves=0, seconds=2.32
Episode 500: score = 1148.0, terminal = 1, updates=30, max_tile=128, valid_moves=122, invalid_moves=0, seconds=2.74


In [ ]:
test_df

In [ ]:
plt.plot(range(len(test_df.max_tiles)),test_df.max_tiles, color = 'b');

In [8]:
%tensorboard --logdir $logs_directory

# Default hyper params

In [ ]:
#environment = Environment.create(
#    environment='gym', level='2048-v0', max_episode_timesteps=1000
#)

# Defaut hyperparams
#agent = Agent.create(
#    agent='dqn',
#    batch_size=16, # Required by Tensorforce
#    update_frequency=4, # Update frequency, TensorForce default : batch_size * 0.25
#    learning_rate = 0.001, # (TensorForce default)
#    discount = 0.99, # (TensorForce default)
#    memory=10000,
#    exploration=0.1, # (0 is the TensorForce default)
#    target_sync_frequency=4, # (1 is the TensorForce default)
#)
#
#for episode in range(1000):
#    train...
#    log2_reward = reward if reward <= 0 else np.log2(reward)